# Test 10
- Random forest
- Removing port longitude and port latitude
- Adding five day sliding window

Adding next:
- Area covered
- 


In [49]:
import pandas as pd
import xgboost as xgb
import numpy as np
import seaborn as sns
from sklearn.model_selection import KFold, cross_val_score, RandomizedSearchCV
import matplotlib.pyplot as plt

In [50]:
train = pd.read_csv('ais_train.csv', sep='|')
train['time'] = pd.to_datetime(train['time'])
train.info()
missing_values = train.isnull().sum()
print("Number of missing values in each column:\n", missing_values)

# Print the columns with missing values
# missing_columns = missing_values[missing_values > 0].index.tolist()
# print("Columns with missing values:", missing_columns)

train.head()

test = pd.read_csv('ais_test.csv', sep=',')
test['time'] = pd.to_datetime(test['time'])
test.head()

vessels = pd.read_csv('vessels.csv', sep='|')
vessels.head()

ports = pd.read_csv('ports.csv', sep='|')
ports.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1522065 entries, 0 to 1522064
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   time       1522065 non-null  datetime64[ns]
 1   cog        1522065 non-null  float64       
 2   sog        1522065 non-null  float64       
 3   rot        1522065 non-null  int64         
 4   heading    1522065 non-null  int64         
 5   navstat    1522065 non-null  int64         
 6   etaRaw     1522065 non-null  object        
 7   latitude   1522065 non-null  float64       
 8   longitude  1522065 non-null  float64       
 9   vesselId   1522065 non-null  object        
 10  portId     1520450 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(3), object(3)
memory usage: 127.7+ MB
Number of missing values in each column:
 time            0
cog             0
sog             0
rot             0
heading         0
navstat         0
etaRaw          0

,portId,name,portLocation,longitude,latitude,UN_LOCODE,countryName,ISO
0,61d36ed80a1807568ff9a064,Port of Algiers,Algiers,3.067222,36.773611,DZALG,Algeria,DZ
1,61d36ed80a1807568ff9a065,Port of Annaba,Annaba,7.772500,36.900556,DZAAE,Algeria,DZ
2,61d36edf0a1807568ff9a070,Port of Oran,Oran,-0.639722,35.712222,DZORN,Algeria,DZ
3,61d36ee00a1807568ff9a072,Port of Skikda,Skikda,6.905833,36.887500,DZSKI,Algeria,DZ
4,61d36ee10a1807568ff9a074,Port of Pago-Pago,Pago-Pago,-170.690556,-14.274167,ASPPG,American Samoa,AS


In [51]:
port_id_mapping = {port_id: idx for idx, port_id in enumerate(train['portId'].unique())}
train['portId'] = train['portId'].map(port_id_mapping)
ports['portId'] = ports['portId'].map(port_id_mapping)


vessel_id_mapping = {vessel_id: idx for idx, vessel_id in enumerate(train['vesselId'].unique())}
train['vesselId'] = train['vesselId'].map(vessel_id_mapping)
vessels['vesselId'] = vessels['vesselId'].map(vessel_id_mapping)
test['vesselId'] = test['vesselId'].map(vessel_id_mapping)

# Create a DataFrame to visualize the vessel ID mapping
vessel_id_mapping_df = pd.DataFrame(list(vessel_id_mapping.items()), columns=['Original Vessel ID', 'Mapped Vessel ID'])


shipping_line_id_mapping = {shipping_line_id: idx for idx, shipping_line_id in enumerate(vessels['shippingLineId'].unique())}
vessels['shippingLineId'] = vessels['shippingLineId'].map(shipping_line_id_mapping)



In [52]:
train = train.merge(ports[['portId', 'latitude', 'longitude']], how='left', left_on='portId', right_on='portId', suffixes=('', '_port'))
train.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,latitude_port,longitude_port
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,0,0,-33.5875,-71.618889
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,1,1,8.9670,-79.533000
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,2,2,39.2325,-76.558889
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,3,3,-34.4625,150.899444
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,4,4,35.7830,-5.817000


In [72]:
import pandas as pd

def create_five_day_windows(df):
    # Ensure that 'time' column is in datetime format
    df['time'] = pd.to_datetime(df['time'])
    
    # Sort data to ensure time sequence within each vessel
    df = df.sort_values(by=['vesselId', 'time']).reset_index(drop=True)
    
    window_size_days = 5
    windows = []

    for vessel_id, group in df.groupby('vesselId'):
        group = group.sort_values(by='time')
        unique_dates = group['time'].dt.date.unique()

        for start_idx in range(len(unique_dates) - window_size_days + 1):
            start_date = unique_dates[start_idx]
            end_date = unique_dates[start_idx + window_size_days - 1]
            window = group[(group['time'].dt.date >= start_date) & (group['time'].dt.date <= end_date)]

            # Skip empty windows
            if len(window) == 0:
                continue

            # Reference row: the first row in the window
            reference_row = window.iloc[0]

            # Append each row in this window with reference features from the first row
            for _, row in window.iterrows():
                windows.append({
                    'vesselId': row['vesselId'],
                    'time': row['time'],
                    'latitude': row['latitude'],
                    'longitude': row['longitude'],
                    'cog': row['cog'],
                    'sog': row['sog'],
                    'rot': row['rot'],
                    'under_way': row['under_way'],
                    # Features based on the first row in the window
                    'latitude_first': reference_row['latitude'],
                    'longitude_first': reference_row['longitude'],
                    'cog_first': reference_row['cog'],
                    'sog_first': reference_row['sog'],
                    'rot_first': reference_row['rot'],
                    'under_way_first': reference_row['under_way'],
                    'time_since_start': (row['time'] - reference_row['time']).total_seconds()
                })

    return pd.DataFrame(windows)



In [64]:
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a)) 
    r = 6371  # Radius of Earth in kilometers
    return r * c  # Distance in kilometers

## Feature engineering

In [73]:
def feature_engineering(train):
    train = train.copy()

    # Sort data to ensure time sequence within each vessel
    train = train.sort_values(by=['vesselId', 'time']).reset_index(drop=True)

    # Handle NaNs by filling forward then backward within each vessel's data
    train[['latitude', 'longitude', 'cog', 'time']] = (
        train.groupby('vesselId')[['latitude', 'longitude', 'cog', 'time']].apply(lambda x: x.ffill().bfill())
    ).reset_index(drop=True)

    features = pd.DataFrame()
    features['vesselId'] = train['vesselId']
    features['time'] = train['time']
    features['latitude'] = train['latitude']
    features['longitude'] = train['longitude']
    features['cog'] = train['cog']
    features['sog'] = train['sog']
    features['rot'] = train['rot']
    features['under_way'] = train['navstat'].isin([0, 8]).astype(int)
    
    # Create five-day sliding windows for each vessel
    five_day_windows = create_five_day_windows(features)
    
    # Merge the five-day window features back into the main features DataFrame
    features = features.merge(five_day_windows, on=['vesselId', 'time'], how='left', suffixes=('', '_five_day'))

    # Additional time and angle transformations
    features['month'] = features['time'].dt.month
    features['day'] = features['time'].dt.day
    features['hour'] = features['time'].dt.hour
    features['minute'] = features['time'].dt.minute
    features['cog_rad'] = np.radians(features['cog'].replace(360, np.nan))
    features['cog_sin'] = np.sin(features['cog_rad'])
    features['cog_cos'] = np.cos(features['cog_rad'])

    return features

train_features = feature_engineering(train)

In [82]:
print(train_features.head())


nan_values = train_features.isnull().sum()
print(nan_values)

# Drop the rows with missing values
train_features = train_features.dropna().reset_index(drop=True)

nan_values = train_features.isnull().sum()
print(nan_values)


   vesselId  latitude  longitude    cog   sog  rot  under_way  \
0         0 -34.74370  -57.85130  284.0   0.7    0          1   
1         0 -35.16787  -56.77210   92.8  14.2    0          1   
2         0 -35.16863  -56.63185   90.5  14.3    0          1   
3         0 -35.16805  -56.53190   88.2  14.3    0          1   
4         0 -35.16715  -56.45306   88.3  12.3    0          1   

   latitude_five_day  longitude_five_day  cog_five_day  ...  longitude_first  \
0          -34.74370           -57.85130         284.0  ...         -57.8513   
1          -35.16787           -56.77210          92.8  ...         -57.8513   
2          -35.16863           -56.63185          90.5  ...         -57.8513   
3          -35.16805           -56.53190          88.2  ...         -57.8513   
4          -35.16715           -56.45306          88.3  ...         -57.8513   

   cog_first  sog_first  rot_first  under_way_first  time_since_start  month  \
0      284.0        0.7        0.0              

In [83]:
def find_last_features(features):
    # Assuming that the last row for every vesselId is the most recent
    last_features = features.groupby('vesselId').last().reset_index()
    return last_features

last_features = find_last_features(train_features)
last_features.head()

,vesselId,latitude,longitude,cog,sog,rot,under_way,latitude_five_day,longitude_five_day,cog_five_day,...,longitude_first,cog_first,sog_first,rot_first,under_way_first,time_since_start,month,day,hour,minute
0,0,34.57936,128.99926,221.5,15.5,0,1,34.57936,128.99926,221.5,...,-122.52499,53.8,0.4,0.0,0.0,1340708.0,5,1,12,41
1,1,1.24460,103.39997,305.1,15.7,11,1,1.24460,103.39997,305.1,...,55.06391,81.6,0.1,0.0,0.0,2195733.0,4,30,10,12
2,2,18.13873,-69.74863,176.0,0.4,0,1,18.13873,-69.74863,176.0,...,-74.15090,0.0,0.0,0.0,0.0,603974.0,5,7,23,59
3,3,41.64055,143.29942,87.6,14.4,0,1,41.64055,143.29942,87.6,...,125.83660,60.8,9.7,0.0,1.0,455585.0,4,26,13,29
4,4,26.58710,121.27831,39.1,12.7,0,1,26.58710,121.27831,39.1,...,103.85164,341.0,0.0,0.0,0.0,562766.0,5,7,12,28
